# Problem 2: For a given set of training data examples stored in a .CSV file, implement and demonstrate the Candidate-Elimination algorithm to output a description of the set of all hypotheses consistent with the training examples.

In [1]:
import csv

In [2]:
def more_general(h1, h2):
    more_general
In [14]:
￼
S,G=candidate_elimination(examples)

 G[0]: {('?', '?', '?', '?', '?', '?')}

 S[0]: {('0', '0', '0', '0', '0', '0')}

 G[1]: {('?', '?', '?', '?', '?', '?')}_parts = []
    for x, y in zip(h1, h2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)

In [3]:
# min_generalizations
def fulfills(example, hypothesis):
    ### the implementation is the same as for hypotheses:
    return more_general(hypothesis, example)

def min_generalizations(h, x):#find S
    h_new = list(h)
    for i in range(len(h)):
        if not fulfills(x[i:i+1], h[i:i+1]):
            h_new[i] = '?' if h[i] != '0' else x[i]
    return [tuple(h_new)]

In [4]:
min_generalizations(h=('0', '0'  , 'sunny'), 
                    x=('rainy', 'windy', 'cloudy'))

[('rainy', 'windy', '?')]

In [5]:
def min_specializations(h, domains, x):#g,domains,-ve instance
    results = []
    for i in range(len(h)):
        if h[i] == "?":
            for val in domains[i]:#for each attrib value in domain not present in instance
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != "0":
            h_new = h[:i] + ('0',) + h[i+1:]
            results.append(h_new)
    return results

In [6]:
min_specializations(h=('?', 'x',), 
                    domains=[['a', 'b', 'c'], ['x', 'y']], 
                    x=('b', 'x'))

[('a', 'x'), ('c', 'x'), ('?', '0')]

In [7]:
with open('wsce.csv')  as csvFile:
        examples = [tuple(line) for line in csv.reader(csvFile)]

#examples = [('sunny', 'warm', 'normal', 'strong', 'warm', 'same',True),
# ('sunny', 'warm', 'high', 'strong', 'warm', 'same',True),
# ('rainy', 'cold', 'high', 'strong', 'warm', 'change',False),
# ('sunny', 'warm', 'high', 'strong', 'cool', 'change',True)]

examples

[('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Y'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Y'),
 ('Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'N'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Y')]

In [8]:
def get_domains(examples):
    d = [set() for i in examples[0]]
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]
get_domains(examples)

[['Rainy', 'Sunny'],
 ['Cold', 'Warm'],
 ['High', 'Normal'],
 ['Strong'],
 ['Cool', 'Warm'],
 ['Change', 'Same'],
 ['N', 'Y']]

In [9]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not fulfills(x, s):#not(s must be true for x)
            S.remove(s)
            Splus = min_generalizations(s, x)
            ## keep only generalizations that have a counterpart in G
            S.update([h for h in Splus if any([more_general(g,h) 
                                               for g in G])])
            ## remove hypotheses less specific than any other in S
            S.difference_update([h for h in S if 
                                 any([more_general(h, h1) 
                                      for h1 in S if h != h1])])
    return S

In [10]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if fulfills(x, g):# g is true for x=> hypothesis is incorrect, since x is -ve instance
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            ## keep only specializations that have a conuterpart in S
            G.update([h for h in Gminus if any([more_general(h, s)
                                                for s in S])])
            ## remove hypotheses less general than any other in G
            G.difference_update([h for h in G if 
                                 any([more_general(g1, h) 
                                      for g1 in G if h != g1])])
    return G

In [11]:

In [14]:
￼
S,G=candidate_elimination(examples)

 G[0]: {('?', '?', '?', '?', '?', '?')}

 S[0]: {('0', '0', '0', '0', '0', '0')}

 G[1]: {('?', '?', '?', '?', '?', '?')}def candidate_elimination(examples):
    domains = get_domains(examples)[:-1]
    
    G = set([("?",)*len(domains)])#initial general hypothesis
    S = set([("0",)*len(domains)])#initial specific hypothesis
    i=0
    print("\n G[{0}]:".format(i),G)
    print("\n S[{0}]:".format(i),S)
    for xcx in examples:
        i=i+1
        x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
        if cx=='Y': # x is positive example
            G = {g for g in G if fulfills(x, g)}#remove any general hyp inconsistent with x
            S = generalize_S(x, G, S)
        else: # x is negative example
            S = {s for s in S if not fulfills(x, s)}#remove any specific hyp inconsistent with x
            G = specialize_G(x, domains, G, S)
        print("\n G[{0}]:".format(i),G)
        print("\n S[{0}]:".format(i),S)
       
    enumerateVersionSpace(S, G)
    return S,G

In [12]:
def enumerateHypothesesBetween_s_g(s, g):
    ''' Given s from special boundary S and g from general boundary G, 
        the function enumerates the intermediate hypotheses. 
    '''
    hypotheses = []
    
    for i, constraint in enumerate(g):
        if constraint != s[i]:
            hypothesis = g[:]
            hypothesis[i]=s[i]
            #print("mid hyp",i,"]",hypothesis)
            hypotheses.append(hypothesis)
    return hypotheses

In [13]:
def enumerateVersionSpace(S, G):
    ''' Given Specific boundary S, and general boundary G, the function enumerates
        all the hypotheses in the version space
    '''
    hypotheses = []
    hypotheses += S
    hypotheses += G
    print("Initial Hypothesis",hypotheses)
     #print(s)
    #for i in range(1,len(hypotheses)):
    #print(hypotheses[i])
    for i in range(1,len(hypotheses)):
        inBetweenHypotheses = enumerateHypothesesBetween_s_g(list(s),list(hypotheses[i]))
        hypotheses.extend(inBetweenHypotheses)
    #print(inBetweenHypotheses)
    #print("Hypothesis with duplicates",hypotheses)
 # Remove duplicates   from hypotheses
    setH = set()
    for h in hypotheses:
        setH.add(tuple(h))
    
    # output list of lists rather than tuples   
    ans=[list(x) for x in setH]
    print("Version Space:",ans)

In [14]:
S,G=candidate_elimination(examples)


 G[0]: {('?', '?', '?', '?', '?', '?')}

 S[0]: {('0', '0', '0', '0', '0', '0')}

 G[1]: {('?', '?', '?', '?', '?', '?')}

 S[1]: {('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same')}

 G[2]: {('?', '?', '?', '?', '?', '?')}

 S[2]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[3]: {('?', '?', '?', '?', '?', 'Same'), ('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}

 S[3]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[4]: {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}

 S[4]: {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
Initial Hypothesis [('Sunny', 'Warm', '?', 'Strong', '?', '?'), ('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')]
Version Space: [['Sunny', '?', '?', 'Strong', '?', '?'], ['Sunny', 'Warm', '?', '?', '?', '?'], ['Sunny', '?', '?', '?', '?', '?'], ['?', 'Warm', '?', '?', '?', '?'], ['?', 'Warm', '?', 'Strong', '?', '?'], ['Sunny', 'Warm', '?', 'Strong', '?', '?']]


In [15]:
examples

[('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Y'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Y'),
 ('Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'N'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Y')]

In [16]:
S

{('Sunny', 'Warm', '?', 'Strong', '?', '?')}

In [17]:
G

{('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}